In [ ]:
%cd /ibex/user/slimhy/PADS/code/

In [ ]:
# Add "3DCoMPaT" to the path
import sys
sys.path.append("jupyter/3DCoMPaT/")
from compat3D import ShapeLoader

In [ ]:
"""
Extracting features into HDF5 files for each split.
"""
import argparse
import numpy as np
import torch

import util.misc as misc
import util.s2vs as s2vs

In [ ]:
def get_args_parser():
    parser = argparse.ArgumentParser("Extracting Features", add_help=False)

    # Model parameters
    parser.add_argument(
        "--batch_size",
        default=32,
        type=int,
        help="Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus",
    )
    parser.add_argument(
        "--text_model_name",
        type=str,
        help="Text model name to use",
    )
    parser.add_argument(
        "--ae",
        type=str,
        metavar="MODEL",
        help="Name of autoencoder",
    )
    parser.add_argument(
        "--ae-latent-dim",
        type=int,
        default=512*8,
        help="AE latent dimension",
    )
    parser.add_argument(
        "--ae_pth",
        required=True,
        help="Autoencoder checkpoint"
    )
    parser.add_argument(
        "--point_cloud_size",
        default=2048,
        type=int,
        help="input size"
    )
    parser.add_argument(
        "--fetch_keys",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_embeds",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--intensity_loss",
        action="store_true",
        default=False,
        help="Contrastive edit intensity loss using ground-truth labels.",
    )

    # Dataset parameters
    parser.add_argument(
        "--dataset",
        type=str,
        choices=["graphedits"],
        help="dataset name",
    )
    parser.add_argument(
        "--data_path",
        type=str,
        help="dataset path",
    )
    parser.add_argument(
        "--data_type",
        type=str,
        help="dataset type",
    )
    parser.add_argument(
        "--max_edge_level",
        default=None,
        type=int,
        help="maximum edge level to use",
    )
    parser.add_argument(
        "--device", default="cuda", help="device to use for training / testing"
    )
    parser.add_argument("--seed", default=0, type=int)
    parser.add_argument("--num_workers", default=60, type=int)
    parser.add_argument(
        "--pin_mem",
        action="store_true",
        help="Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.",
    )

    return parser

In [ ]:
# Set dummy arg string to debug the parser
call_string = """--ae_pth ckpt/ae_m512.pth \
    --ae kl_d512_m512_l8 \
    --ae-latent-dim 4096 \
    --batch_size 32 \
    --num_workers 8 \
    --device cuda"""
    

# Parse the arguments
args = get_args_parser()
args = args.parse_args(call_string.split())

In [ ]:
# Set device and seed
device = torch.device(args.device)
misc.set_all_seeds(args.seed)
torch.backends.cudnn.benchmark = True

# Instantiate autoencoder
ae = s2vs.load_model(args.ae, args.ae_pth, device, torch_compile=True)
ae = ae.eval()

In [ ]:
import zipfile
from metadata import SHAPENET_CLASSES, COMPAT_CLASSES, COMPAT_TRANSFORMS


ACTIVE_CLASS = "chair"
METADATA_DIR = "/ibex/user/slimhy/3DCoMPaT/3DCoMPaT-v2/metadata"
ZIP_SRC = "/ibex/user/slimhy/surfaces.zip"
ZIP_PATH = "/ibex/user/slimhy/3DCoMPaT/3DCoMPaT_ZIP.zip"
N_POINTS = 2**21


def shapenet_iterator(shape_cls):
    # List all files in the zip file
    with zipfile.ZipFile(ZIP_SRC, 'r') as zip_ref:
        files = zip_ref.namelist()

        for file in files:
            if not file.startswith(SHAPENET_CLASSES[shape_cls]): continue
            if not file.endswith(".npz"): continue
            # Read a specific file from the zip file
            with zip_ref.open(file) as file:
                data = np.load(file)
                yield data["points"].astype(np.float32)
        

def compat_iterator(shape_cls):
    train_dataset = ShapeLoader(
        zip_path=ZIP_PATH,
        meta_dir=METADATA_DIR,
        split="train",
        n_points=N_POINTS,
        shuffle=True,
        seed=0,
        filter_class=COMPAT_CLASSES[shape_cls]
    )

    for shape_id, shape_label, pointcloud, point_part_labels in train_dataset:
        yield COMPAT_TRANSFORMS[shape_cls](pointcloud)

In [ ]:
from shapeloaders import SingleManifoldDataset

OBJ_DIR = "/ibex/user/slimhy/PADS/data/obj_manifold/"
OBJ_ID = 2

# Instantiate the dataset
shape_dataset = SingleManifoldDataset(
    OBJ_DIR,
    ACTIVE_CLASS,
    N_POINTS,
    normalize=False,
    sampling_method="volume+near_surface",
    contain_method="occnets",
    decimate=True,
    sample_first=False
)
it_shape = shape_dataset[OBJ_ID]

# Initialize the latents
orig_dataset = SingleManifoldDataset(
    OBJ_DIR,
    ACTIVE_CLASS,
    N_POINTS,
    normalize=False,
    sampling_method="surface"
)
surface_points, _ = next(orig_dataset[OBJ_ID])
init_latents = s2vs.encode_pc(ae, surface_points).detach()

In [ ]:
from schedulefree import AdamWScheduleFree
import torch.nn.functional as F


def optimize_latents(ae, shape_it, init_latents, max_iter=100, optimizer=None):
    """
    Optimize input latent codes w.r.t. a single object.
    """
    # Activate gradient computation for the latents
    latents = init_latents.clone().detach().to(device).requires_grad_(True)
    if optimizer is None:
        optimizer = torch.optim.Adam
    optimizer = optimizer([latents], lr=1e-3)
    
    # Main optimization loop
    for i, (surface_points, occs) in enumerate(shape_it):
        optimizer.zero_grad()

        # Sample points from the shape surface
        surface_points = surface_points.unsqueeze(0).to(device)

        # Query the autoencoder
        logits = s2vs.query_latents(ae, latents, surface_points, use_graph=True).squeeze()
        occs = occs.float().to(device)

        # Compute BCE loss
        loss = F.binary_cross_entropy_with_logits(logits, occs).mean()
        
        # Add gradient clipping
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print("Iter %d: Loss %.4f" % (i, loss.item()))
        if i >= max_iter:
            break

    return latents.detach().cpu()

optimized_latents = optimize_latents(ae,
                                     it_shape,
                                     init_latents,
                                     max_iter=20,
                                     optimizer=AdamWScheduleFree)

In [ ]:
from eval.metrics import iou_occupancies, chamfer_distance, f_score
from util.sampling import sample_surface_tpp
from util.contains.inside_mesh import is_inside
from util.misc import d_GPU, show_side_by_side, timer_init, timer_end


def evaluate_reconstruction(obj_original, rec_mesh, latents, n_queries=2**21, n_queries_chamfer=2**15, query_method="occnets"):
    """
    Compute IoU and Chamfer distance between the original object and the reconstructed mesh.
    - IoU: Computed using [n_queries] random points sampled from the 3D space.
    - CD: Computed using [n_queries] points sampled from the object surface.
    """
    timer_init("surface_sampling")

    # Sample N_POINTS points from obj_original and rec_mesh
    pc_original = sample_surface_tpp(obj_original, n_queries_chamfer)
    pc_rec = sample_surface_tpp(rec_mesh, n_queries_chamfer)
    chamfer = chamfer_distance(pc_original, pc_rec)
    f_sc = f_score(gt=pc_original, pred=pc_rec)
    
    timer_end()
    timer_init("point_sampling")

    # Sample n_queries points from the 3D space
    point_queries = torch.rand((1, n_queries, 3)).cuda()
    
    timer_end()
    timer_init("predict_occ")
    
    # Get predicted occupancies for each point
    latents = latents.to(device)
    pred_occs = s2vs.predict_occupancies(ae, latents, point_queries, n_queries)

    timer_end()
    timer_init("is_inside")

    # Get ground-truth occupancy for each point
    gt_occs = is_inside(obj_original, point_queries, query_method=query_method)

    timer_end()
    timer_init("iou_occ")

    # Compute IoU between predicted and ground-truth occupancies
    iou = iou_occupancies(pred_occs, gt_occs)
    
    timer_end()
    
    return f_sc.item(), chamfer.item(), iou.item()

In [ ]:
# Decode the optimized latents
rec_mesh = s2vs.decode_latents(ae, d_GPU(optimized_latents), grid_density=256, batch_size=128**3)

In [ ]:
# Decode the optimized latents
rec_mesh = s2vs.decode_latents(ae, d_GPU(optimized_latents), grid_density=256, batch_size=128**3, smooth_volume=True)

In [ ]:
orig_mesh = shape_dataset.obj
evaluate_reconstruction(orig_mesh, rec_mesh, optimized_latents, query_method="occnets")

In [ ]:
# Decode the old latents
old_latents = init_latents
old_rec_mesh = s2vs.decode_latents(ae, d_GPU(old_latents), grid_density=256, batch_size=128**3)

In [ ]:
old_rec_mesh = s2vs.decode_latents(ae, d_GPU(old_latents), grid_density=256, batch_size=128**3, smooth_volume=True)

In [ ]:
evaluate_reconstruction(orig_mesh, old_rec_mesh, old_latents, query_method="occnets")

In [ ]:
show_side_by_side(orig_mesh, rec_mesh, old_rec_mesh)

In [ ]:
import numpy as np
import mcubes

# Create a data volume (30 x 30 x 30)
X, Y, Z = np.mgrid[:30, :30, :30]
u = (X-15)**2 + (Y-15)**2 + (Z-15)**2 - 8**2

# Extract the 0-isosurface
vertices, triangles = mcubes.marching_cubes(u, 0)

# Export the result to sphere.dae
mcubes.export_mesh(vertices, triangles, "sphere.dae", "MySphere")
